In [1]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [2]:
import braindecode
import numpy as np
from sklearn.model_selection import train_test_split

# load nn model from braindecode
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

import core.data as data

%load_ext autoreload
%autoreload 2

In [7]:
# load data 
x_loaded, y_loaded = data.load_x('data/x_train.h5'), data.load_y('data/y_train.csv')[:, 1]

Started loading file data/x_train.h5
Finished loading the file.
Started loading file data/y_train.csv
Finished loading the file.


In [8]:
# convert y to categorical with np.eye(d)[y_loaded]
# and flatten the 40 independent samples
x, y = data.flatten_x(x_loaded), data.flatten_y(np.eye(2)[y_loaded], repeat=40)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.66)
x_train, x_test = x_train.squeeze(), x_test.squeeze() 

# Only one value : 0 or 1  
y_train, y_test = np.argmax(y_train, axis=1), np.argmax(y_test, axis=1)
print('shape : ', x_train.shape, x_test.shape, y_train.shape, y_test.shape)

shape :  (24974, 7, 500) (12866, 7, 500) (24974,) (12866,)


In [9]:
# define the model

cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = x_train.shape[1]
# final_conv_length = auto ensures we only get a single output in the time dimension
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=x_train.shape[2],
                        final_conv_length='auto')

from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
#optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1,)

In [10]:
model.fit(x_train, y_train, epochs=5, batch_size=64, scheduler='cosine',
         validation_data=(x_test, y_test),)

2020-02-07 20:04:19,289 INFO : Run until first stop...
2020-02-07 20:04:47,298 INFO : Epoch 0
2020-02-07 20:04:47,299 INFO : train_loss                2.64547
2020-02-07 20:04:47,301 INFO : valid_loss                2.65069
2020-02-07 20:04:47,302 INFO : train_misclass            0.73452
2020-02-07 20:04:47,304 INFO : valid_misclass            0.73993
2020-02-07 20:04:47,305 INFO : runtime                   0.00000
2020-02-07 20:04:47,307 INFO : 
2020-02-07 20:06:23,473 INFO : Time only for training updates: 96.16s
2020-02-07 20:06:56,814 INFO : Epoch 1
2020-02-07 20:06:56,815 INFO : train_loss                1.23345
2020-02-07 20:06:56,818 INFO : valid_loss                1.21751
2020-02-07 20:06:56,819 INFO : train_misclass            0.23356
2020-02-07 20:06:56,821 INFO : valid_misclass            0.22929
2020-02-07 20:06:56,824 INFO : runtime                   124.18296
2020-02-07 20:06:56,828 INFO : 
2020-02-07 20:08:44,797 INFO : Time only for training updates: 107.96s
2020-02-07